## Read Data

In [1]:
import numpy as np
import pandas as pd
import spacy
import io
import glob

filenames = glob.glob('./Data/*txt')
file_name_and_text = {}

for file in filenames:
    with io.open(file, mode="r", encoding="utf-8") as target_file:
         file_name_and_text[file] = target_file.read()

df = pd.DataFrame.from_dict(file_name_and_text, orient='index')
df.reset_index(inplace = True)
df.columns = ['file_name', 'text']
df

,file_name,text
0,./Data\09415616C1 HKR vágott.txt,kibővült az eddigi szolgáltatási összeg az Aeg...
1,./Data\111951940C1 vágott.txt,Üdvözlöm kereslek Jó napot kívánok Én ready fi...
2,./Data\112905428C1 GB vágott.txt,örököltem egy lakást Vagyis te feleségemre van...
3,./Data\113251125C1 LB vágott.txt,Miben segíthetek önnek Érdeklődni szeretnék ho...
4,./Data\123615343C1 Timó vágott.txt,Na vagyok és kaptam kötöttem önöknél egy gépjá...


# Tokenization and Lemmatization with Spacy

### Installing Spacy  
conda install -c conda-forge spacy  
conda install -c conda-forge spacy-lookups-data


### Download Custom Language Model
conda install pip  
pip install https://github.com/oroszgy/spacy-hungarian-models/releases/download/hu_core_ud_lg-0.3.1/hu_core_ud_lg-0.3.1-py3-none-any.whl  


In [2]:
import hu_core_ud_lg
nlp = hu_core_ud_lg.load()

#### Add and Remove Stop Words

In [14]:
nlp.vocab["nem"].is_stop = False
nlp.Defaults.stop_words

{'a',
 'abban',
 'ahhoz',
 'ahogy',
 'ahol',
 'aki',
 'akik',
 'akkor',
 'akár',
 'alatt',
 'amely',
 'amelyek',
 'amelyekben',
 'amelyeket',
 'amelyet',
 'amelynek',
 'ami',
 'amikor',
 'amit',
 'amolyan',
 'amíg',
 'annak',
 'arra',
 'arról',
 'az',
 'azok',
 'azon',
 'azonban',
 'azt',
 'aztán',
 'azután',
 'azzal',
 'azért',
 'be',
 'belül',
 'benne',
 'bár',
 'cikk',
 'cikkek',
 'cikkeket',
 'csak',
 'de',
 'e',
 'ebben',
 'eddig',
 'egy',
 'egyes',
 'egyetlen',
 'egyik',
 'egyre',
 'egyéb',
 'egész',
 'ehhez',
 'ekkor',
 'el',
 'ellen',
 'elo',
 'eloször',
 'elott',
 'elso',
 'elég',
 'előtt',
 'emilyen',
 'ennek',
 'erre',
 'ez',
 'ezek',
 'ezen',
 'ezt',
 'ezzel',
 'ezért',
 'fel',
 'felé',
 'ha',
 'hanem',
 'hiszen',
 'hogy',
 'hogyan',
 'hát',
 'ide',
 'igen',
 'ill',
 'ill.',
 'illetve',
 'ilyen',
 'ilyenkor',
 'inkább',
 'is',
 'ismét',
 'ison',
 'itt',
 'jobban',
 'jó',
 'jól',
 'kell',
 'kellett',
 'keressünk',
 'keresztül',
 'ki',
 'kívül',
 'között',
 'közül',
 'le',
 '

### Preprocessing

In [17]:
# Add custom stop words
# nlp_hu.Defaults.stop_words |= {'', ''} -- add before lemmatization
extra_stops = ['\n ', '\n', 'ugye', 'na', 'lehet', 'ön', 'ok', 'oké'] # add after lemmatization

In [18]:
calls = df['text']

tokenized_calls = []
for call in calls:
    doc = nlp(call.lower())
    text = []
    for word in doc:
        if not word.is_stop and not word.is_punct and not word.like_num and word.lemma_ not in extra_stops:
            text.append(word.lemma_)
    tokenized_calls.append(text)

df['text_tokenized'] = tokenized_calls
df

,file_name,text,text_tokenized
0,./Data\09415616C1 HKR vágott.txt,kibővült az eddigi szolgáltatási összeg az Aeg...,"[kibővül, eddigi, szolgáltatási, összeg, aegon..."
1,./Data\111951940C1 vágott.txt,Üdvözlöm kereslek Jó napot kívánok Én ready fi...,"[üdvözl, keres, nap, kíván, ready, fizetés, ka..."
2,./Data\112905428C1 GB vágott.txt,örököltem egy lakást Vagyis te feleségemre van...,"[örököl, lakás, feleségem, lakás, családi, ott..."
3,./Data\113251125C1 LB vágott.txt,Miben segíthetek önnek Érdeklődni szeretnék ho...,"[miben, segíthet, érdeklődik, szeret, kap, lev..."
4,./Data\123615343C1 Timó vágott.txt,Na vagyok és kaptam kötöttem önöknél egy gépjá...,"[kap, köt, gépjármű, felelősségbiztosítás, kap..."


# Vectorizing

#### CountVectorizer 1-gram

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

def dummy_fun(doc):
    return doc

cnts = CountVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)

cnts.fit(tokenized_calls)
cnts.vocabulary_

{'kibővül': 243,
 'eddigi': 78,
 'szolgáltatási': 450,
 'összeg': 555,
 'aegon': 9,
 'orvosi': 398,
 'tápszer': 487,
 'gyűrű': 170,
 'felügyelet': 146,
 'bizonyára': 47,
 'tetszik': 477,
 'lát': 304,
 'na': 366,
 'gond': 165,
 'fia': 149,
 'sor': 432,
 'kap': 224,
 'ki': 241,
 'fordul': 157,
 'telefon': 471,
 'ügyelet': 564,
 'kapcsán': 227,
 'probléma': 409,
 'szép': 454,
 'elmagyaráz': 100,
 'hölgy': 193,
 'december': 67,
 'fog': 154,
 'megkap': 318,
 'mennyi': 344,
 'díj': 73,
 'kér': 254,
 'máris': 358,
 'nézemne': 390,
 'haragszik': 175,
 'várakoztató': 517,
 'n50': 365,
 'néz': 389,
 'levelő': 301,
 'tájékoztatás': 485,
 'inka': 206,
 't': 460,
 'legalábbis': 294,
 'vonatkozó': 514,
 'hozzájárul': 187,
 'nehez': 373,
 'szolgáltatás': 449,
 'megígér': 336,
 'megkötött': 323,
 'tök': 492,
 'jóorvosi': 221,
 'postai': 408,
 'út': 559,
 'tud': 481,
 'ön': 552,
 'küld': 278,
 'email': 116,
 'továbbít': 480,
 'ugyanúgy': 499,
 'megfelelő': 313,
 'nagyszerű': 368,
 'nyugodt': 381,
 'ele

In [33]:
vectors = cnts.transform(tokenized_calls).todense()
vectors.shape

(5, 573)

In [41]:
vectors

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 1, 0],
        [0, 2, 0, ..., 0, 0, 0],
        [0, 3, 0, ..., 1, 4, 2],
        [1, 2, 1, ..., 0, 1, 0]], dtype=int64)

In [50]:
vocab = np.array(cnts.get_feature_names())
vocab.shape

(572,)

In [53]:
vocab[350:400]

array(['mindegy', 'mindenképpen', 'mire', 'mond', 'mondható', 'mostani',
       'mostantól', 'motoromm', 'máris', 'márpedig', 'második', 'módosul',
       'módosít', 'módosítás', 'működik', 'n50', 'na', 'nagyapa',
       'nagyszerű', 'nagyvilág', 'nap', 'negyedév', 'negyedéves', 'nehez',
       'nek', 'nektaria', 'netrisken', 'nincsen', 'nyilván', 'nyolc',
       'nyomtat', 'nyugodt', 'nyújt', 'nál', 'nála', 'nálam', 'négy',
       'négyes', 'név', 'néz', 'nézemne', 'nő', 'ok', 'oké', 'olcsó',
       'oldal', 'oly', 'om', 'orvosi', 'os'], dtype='<U20')

#### CountVectorizer 2-gram

In [52]:
from sklearn.feature_extraction.text import CountVectorizer

def dummy_fun(doc):
    return doc

cnts_2gram = CountVectorizer(
    analyzer='word',
    ngram_range=(2, 2),
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)

cnts_2gram.fit(tokenized_calls)
cnts_2gram.vocabulary_

{'kibővül eddigi': 422,
 'eddigi szolgáltatási': 132,
 'szolgáltatási összeg': 805,
 'összeg aegon': 1018,
 'aegon orvosi': 15,
 'orvosi tápszer': 697,
 'tápszer gyűrű': 909,
 'gyűrű felügyelet': 289,
 'felügyelet bizonyára': 227,
 'bizonyára tetszik': 75,
 'tetszik lát': 864,
 'lát na': 549,
 'na gond': 640,
 'gond fia': 276,
 'fia sor': 231,
 'sor kap': 759,
 'kap ki': 383,
 'ki fordul': 420,
 'fordul telefon': 262,
 'telefon ügyelet': 853,
 'ügyelet kapcsán': 1039,
 'kapcsán probléma': 399,
 'probléma szép': 722,
 'szép elmagyaráz': 812,
 'elmagyaráz hölgy': 159,
 'hölgy december': 323,
 'december fog': 113,
 'fog kap': 253,
 'kap na': 388,
 'na megkap': 643,
 'megkap mennyi': 570,
 'mennyi díj': 602,
 'díj kap': 125,
 'fordul kér': 260,
 'kér máris': 447,
 'máris nézemne': 630,
 'nézemne haragszik': 681,
 'haragszik várakoztató': 294,
 'várakoztató n50': 951,
 'n50 néz': 639,
 'néz máris': 679,
 'máris levelő': 629,
 'levelő tájékoztatás': 533,
 'tájékoztatás inka': 906,
 'inka t':